# LOGISTIC SYSTEM 

## Authors: 
* Ana Laura 
* @flima21
* Murilo Araujo


Link to document firebase: https://www.c-sharpcorner.com/article/firebase-crud-operations-using-python/

In [ ]:
!pip install python_jwt
!pip install gcloud 
!pip install sseclient
!pip install pycrypto
!pip install requests-toolbelt
!pip install firebase
!pip install firebase-admin
!pip install pyrebase

In [ ]:
# This cell will be all the imports of the system
import json
import time
import geopy.distance as geo 

from datetime import datetime, timedelta
from firebase import firebase
from tqdm import tqdm

## Functions



In [ ]:
# Filter the json that get the firebase
def filter_json_dict(file_json_dict,key,value_search):
    if isinstance(file_json_dict,dict):
        dictionary = dict()
        key_dict = 0

        for k in file_json_dict:
            search = file_json_dict[k][key]
            
            # If value search is the same in key, the script return
            if search == value_search:
                dictionary[key_dict] = file_json_dict[k]
                key_dict += 1

        return dictionary
    else:
        json_file = []

        for k in range(len(file_json_dict)):
            search = file_json_dict[k][key]

            if search == value_search:
               json_file.append(file_json_dict[k])

               return json_file

# Get the last id of order
def get_last_id(obj,key_search):
    max_value = 0
    
    if isinstance(obj,dict):
        for key in obj:
            if key:
                id = obj[key][key_search]

                if id >= max_value:
                    max_value = id 
    else:
        for key in range(len(obj)):
            if key:
                id = obj[key][key_search]

                if id > max_value:
                    max_value = id 

    return max_value

def get_distance_between_points(client_1,client_2):
	return geo.distance(client_1,client_2).km

# Verify the index if exists correlation 
def insert_order_client(connection,database_file_client,document_client,items,delivery_date=None):
    
    if not delivery_date:
        delivery_date = str(datetime.today().date())

    delivery_date = datetime.strptime(delivery_date,'%Y-%m-%d').date()
    
    # If the date order is lower than today, the order will not insert
    if int(time.mktime(delivery_date.timetuple())) < int(time.mktime(datetime.today().date().timetuple())):
        print('Order is not allow to insert because the that given is lower than today.')

        return False 
    
    if not document_client:
        print('Its necessary the document client to search, please give')
        
        return False

    # Search for the client
    client_order = filter_json_dict(database_file_client,'nr_cpf_cnpj',document_client)

    # If len zero means that the client not exists
    if client_order == None:
        print('Order is not allow to insert because the client given not exists.')
        
        return False

    # If len zero means that this not items
    if len(items) == 0:
        print('To insert order is necessary insert items')

        return False 

    order = connection.get('fat_pedido','')

    # Get tha last id order
    new_id = get_last_id(order,'id_pedido') + 1

    new_data_order = {
        'dt_entrega': delivery_date,
        'id_cliente':client_order[0]['id_cliente'],
        'id_pedido':new_id,
        'id_status': 5
    }

    
    result_to_insert = connection.post('fat_pedido',new_data_order)

    if len(result_to_insert) > 0:
        products = json.loads(json.dumps(connection.get('cad_produto','')))
        order_item = json.loads(json.dumps(connection.get('fat_pedido_item','')))

        for item in items:
            for product in products:
                product_exist = product['cd_produto']
                
                # If product exist is equal item search in order, means that product exists in database
                if product_exist == str(item):
                    new_data_order_item = {
                        'id_pedido_item': get_last_id(order_item,'id_pedido_item') + 1,
                        'id_pedido':new_id,
                        'id_produto': product['id_produto']
                    }

                    connection.post('fat_pedido_item',new_data_order_item)
            
        print('The order was sent to Delivery Department')

        return True
        
    else:
        print('Record not insert')

        return False

# Get the id status of the delivery
def get_id_status(connection,code):
    status = json.loads(json.dumps(connection.get('cad_status_entrega','')))

    filter_status = filter_json_dict(status,'cd_status',code)

    if filter_status:
        return filter_status[0]['id_status']
    else:
        return None

# Get all the orders to ready and with date today to delivery
def get_orders_delivery(connection,date_delivery=None):
    
    if not date_delivery:
        date_delivery = str(datetime.today().date())

    prepare_orders = []

    id_status = get_id_status(connection,'P')

    order_obj = json.loads(json.dumps(connection.get('fat_pedido','')))

    # Filter to date 
    order = filter_json_dict(order_obj,'dt_entrega',date_delivery)
    # Filter by status
    order = filter_json_dict(order,'id_status',id_status)

    if order == None:
        print('Is not order to delivery today.')

        return False 
    
    orders_key = []

    for keys_order in order_obj:
        
        if (order_obj[keys_order]['dt_entrega'] == date_delivery) and (order_obj[keys_order]['id_status'] == id_status):
            orders_key.append(keys_order)
  
    print(f'There are {len(order)} to delivery today.')

    return order, orders_key

# Set the order the routing to delivery in the client 
def set_routing_order(connection,orders,initial_point):
    loading = []
    
    clients = json.loads(json.dumps(connection.get('cad_cliente','')))
    item_order = json.loads(json.dumps(connection.get('fat_pedido_item','')))
    product = json.loads(json.dumps(connection.get('cad_produto','')))

    client_ids = []

    for order in orders:
        client_ids.append(orders[order]['id_cliente'])

    client_ids = [*set(client_ids)]

    for ids in range(len(client_ids)):
        id = client_ids[ids]

        # INFORMATION ABOUT CLIENT 
        client_id = id
        
        client = filter_json_dict(clients,'id_cliente',client_id)

        client_name = client[0]['nm_cliente']
        client_doc = client[0]['nr_cpf_cnpj']
        client_long = client[0]['longitude']
        client_lat = client[0]['latitude']
        client_street = client[0]['ds_logradouro']
        client_city = client[0]['cd_municipio']
        client_zip = client[0]['nr_cep']

        # Information about order
        orders_client = filter_json_dict(orders,'id_cliente',id)

        qt_order = len(orders_client)
        
        date_delivery = orders_client[0]['dt_entrega']

        # information about items 
        vl_volume_total = 0
        vl_weight_total = 0

        order_ids = []

        for peds in orders_client:
            order_ids.append(orders_client[peds]['id_pedido'])

        for i in range(len(order_ids)):
            items = filter_json_dict(item_order,'id_pedido',order_ids[i])

            if items == None:
                continue 

            for item in items:
                info_product = filter_json_dict(product,'id_produto',(items[item]['id_produto']))

                if info_product[0]['qt_disponivel'] <= 0:
                    nm_produto = info_product[0]['nm_produto']

                    print(f'The product {nm_produto} is not enough in stock')

                    continue

                vl_volume_total += (info_product[0]['vl_volume'])
                vl_weight_total += (info_product[0]['vl_peso'])            

        dictionarie = {
            'client_name': client_name,
            'doc_client':client_doc,
            'zip_client':client_zip,
            'street':client_street,
            'city':client_city,
            'delivery_date':date_delivery,
            'qt_order':qt_order,
            'vl_total_volume':vl_volume_total,
            'vl_total_weight':vl_weight_total,
            'lat':client_lat,
            'long':client_long,
        }

        loading.append(dictionarie)

    sequence_delivery = []
    max_distance = 0
    min_distance = 0

    lk = LinkedList() # The representation of the delivery
    km_total = 0

    # Bellow is the organization data
    for routings in range(len(loading)):
        
        coords_client = (loading[routings]['long'],loading[routings]['lat'])

        next_point = get_distance_between_points(initial_point,coords_client)
        km_total += next_point

        if next_point > max_distance:
            max_distance = next_point 
            sequence_delivery.append(loading[routings]['doc_client'])
        else:
            min_distance = next_point 
            sequence_delivery.insert(0,loading[routings]['doc_client'])
        
    lista_reverse = sequence_delivery[::-1]

    for clients_sequence in range(len(lista_reverse)):
        lk.insert_at_beginning(lista_reverse[clients_sequence])

    print('\nRoute ready to delivery\n')
    return lk, loading,int(km_total)

## Class

### Stack

In [ ]:
class Stack:
    def __init__(self):
        self.elements = []

    def push(self, data):
        self.elements.append(data)

    def pop(self):
        return self.elements.pop()

    def peek(self):
        return self.elements[len(self.elements)-1]

    def size(self):
        return len(self.elements)

    def isEmpty(self):
        return self.elements == []

### Queue

In [ ]:
class Queue:
    def __init__(self):
        self.items = []
    
    def isEmpty(self):
        return self.items == []
    
    def enqueue(self, item):
        self.items.insert(0, item)
    
    def dequeue(self):
        return self.items.pop()
    
    def size(self):
        return len(self.items)

    def search(self,value_search):
        
        for i in range(self.size()):
            if self.items[i] == value_search:
                return value_search

        return False

### LinkedList

In [ ]:
class Node:

    def __init__(self, data=None, next=None):
        self.data = data
        self.next = next

    def __repr__(self):
        return '%s -> %s' % (self.data, self.next)
    
class LinkedList:
  
    def __init__(self):
        self.root = None

    def __repr__(self):
        return '[' + str(self.root) + ']'

    def search(self, data):
        currently = self.root 

        while currently and currently.data != data:
            currently = currently.next
        
        return currently

    def insert_at_beginning(self, new_data):
        new_node = Node(new_data)

        new_node.next = self.root

        self.root = new_node

    def insert_after_node(self, previous_node, new_data):

        new_node = Node(new_data)

        new_node.next = previous_node.next

        previous_node.next = new_node
    
    def size(self):
        head = self.root 

        value = 0

        while head:
            value += 1

            head = head.next 
        
        return value 

    def sum_all_elements_odd(self):
        head = self.root

        soma = 0 
        while head:
            if head.data % 2 != 0:
                soma += head.data

            head = head.next
        
        return soma

    def sum_all_elements(self):
        head = self.root

        soma = 0 
        while head:
            soma += head.data

            head = head.next
        
        return soma

    def union_linked_list(self,v):
        head_v = v.root

        while head_v:
            self.insert_at_beginning(head_v.data)
            
            head_v = head_v.next

    def intersection_linked_list(self,v):
        lk = LinkedList()

        qt_node_self = self.size()
        qt_node_v = v.size()

        head_self = self.root 
        head_v = v.root

        if qt_node_self >= qt_node_v:

            while head_self:
                values = v.search(head_self.data)

                if values:
                    lk.insert_at_beginning(values.data)

                head_self = head_self.next
        else:
            while head_v:
                values = self.search(head_v.data)

                if values:
                    lk.insert_at_beginning(values.data)

                head_v = head_v.next

        return lk

    def diff_linked_list(self,v):
        lk = LinkedList()

        head_self = self.root 

        while head_self:
            values = v.search(head_self.data)

            if not values:
                lk.insert_at_beginning(head_self.data)

            head_self = head_self.next

        return lk

    def copy(self):
        head = self.root

        lk = LinkedList()

        while head:
            lk.insert_at_beginning(head.data)

            head = head.next 

        return lk 
    def remove(self, data):

        if self.root.data == data:
            self.root = self.root.next
        else:

            previous_node = None
            current_node = self.root

            while current_node and current_node.data != data:
                previous_node = current_node
                current_node = current_node.next
            
            if current_node:
                previous_node.next = current_node.next
            else:
                previous_node.next = None

    def remove_between_values(self,value_a,value_b):
        head = self.root 
        
        while head:
            if head.data >= value_a and head.data <= value_b:
                self.remove(head.data)
            
            head = head.next

            
        

### Truck

In [ ]:
class Truck:

    def __init__(self,type_truck='tk'):
        self.height = 0 # m 
        self.length = 0 # m
        self.deep = 0   # m
        self.orders_inside = Queue()
        self.clients_inside = []
        self.weight = 0
        self.gps = None 
        self.all_info_about_client = None 
        self.items = None 
        self.type_truck = type_truck

        # Type Truck 
        # Tanker = tk 
        # Tractor Trailer = tt 

        if type_truck.lower() == 'tk':
            self.height = 50
            self.length = 10
            self.deep = 20000
            self.weight = 1000

        elif type_truck.lower() == 'tt':
            self.height = 25
            self.length = 5
            self.deep = 10000
            self.weight = 500

        elif type_truck.lower() == 'cc':
            self.height = 25
            self.length = 5
            self.deep = 100000000000
            self.weight = 5000000000
            
        self.volume = (self.height * self.length * self.deep)

    def load_truck_to_delivery(self,connection,obj_client,list_clients,order_values,key_json):
        if list_clients.size() == 0:
            print('The delivery list doenst has clients today.')
            return False

        if len(key_json) == 0:
            print('There are not orders to delivery')


            return False 
            
        container = Queue()

        self.gps = list_clients.copy()

        self.all_info_about_client = order_values

        head = list_clients.root
        
        while head:
            value_per_client = filter_json_dict(order_values,'doc_client',head.data)
            
            if value_per_client == None:
                head = head.next
                continue 

            volume = value_per_client[0]['vl_total_volume']
            weight = value_per_client[0]['vl_total_weight']

            if (volume > self.volume) or (weight > self.weight):
                print('Capactiy maximun not allowed')

                list_clients.remove(head.data)                      

                head = head.next
                
                continue

            container.enqueue(head.data)

            self.volume -= volume
            self.weight -= weight
            self.clients_inside.append(head.data)
            
            head = head.next

        orders = connection.get('fat_pedido', '')
        
        if container.size() > 0:
            for cli in range(container.size()):
                client = filter_json_dict(obj_client,'nr_cpf_cnpj',container.items[cli])

                if client == None:
                    continue 

                id_client = client[0]['id_cliente']

                for key_ped in range(len(key_json)):

                    if orders == None:
                        continue 

                    get_key = orders[key_json[key_ped]]

                    if get_key and orders[key_json[key_ped]]['id_cliente'] == id_client:
                        self.orders_inside.enqueue(key_json[key_ped])

        return container

    def truck_is_load(self):
        return self.orders_inside.size() > 0

    def redefine_truck(self):
        return Truck(self.type_truck)
    
    def simulate_delivery(self,connection):
        print('\nStart routing\n')
        
        for i in range(len(self.clients_inside)):

            client = self.clients_inside[i]
            # print(client)
            search_route = self.gps.search(client)      
            
            if search_route == None:
                continue 

            client_name = self.all_info_about_client[i]['client_name']
            address = str(self.all_info_about_client[i]['street']) + ', ' + str(self.all_info_about_client[i]['zip_client']) + ' ' + 'City: ' + str(self.all_info_about_client[i]['city'])

            print('*'*100)
            print(f'Delivery to client {client_name} located at {address}')

            for i in tqdm(range(100)):
                time.sleep(0.1)

            print('\nSuccessful delivery')
            print('*'*100)
            print('\n')

            self.gps.remove(client)

        print('Updating the database. Waiting seconds.')

        for i in range(self.orders_inside.size()):
            connection.put(f'fat_pedido/{self.orders_inside.items[i]}','id_status',get_id_status(connection,'E')) 

            #print('The order change status')

        self.orders_inside = Queue()

        print('All orders was successfully delivered')

        print('\nThe truck is back of plant')
        return self.redefine_truck()


### Reports

In [ ]:
class Report:
    
    def __init__(self,connection):
        self.connection = connection 

    def get_report_client(self):
        if self.connection == None:
            print('To generate reports its necessary a connection with database')

            return False 

        clients = self.connection.get('cad_cliente', '')

        if clients == None:
            print('There arent clients in database')

            return False 

        print('*'*100)
        print('* REPORT CLIENT IN DATABASE')
        print('*'*100)
        print('\n')

        for client in range(len(clients)):
            print('*'*100)

            name = clients[client]['nm_cliente']
            doc = clients[client]['nr_cpf_cnpj']
            address = str(clients[client]['ds_logradouro']) + ', ' + str(clients[client]['nr_cep']) + ' ' + 'City: ' + str(clients[client]['cd_municipio'])
            coords = (clients[client]['latitude'],clients[client]['longitude'])

            print(f'Name: {name}')    
            print(f'Document: {doc}')  
            print(f'Address: {address}')  
            print(f'Coords: {coords}')  

            print('*'*100)

            print('\n')


    def get_report_order_per_client(self):
        if self.connection == None:
            print('To generate reports its necessary a connection with database')

            return False 

        clients = self.connection.get('cad_cliente', '')

        print('*'*100)
        print('* REPORT CLIENT/PRODUCT IN DATABASE')
        print('*'*100)
        print('\n')

        if clients == None:
            print('There arent clients in database')

            return False 

        for client in range(len(clients)):
            name = clients[client]['nm_cliente']
            id = client + 1
            qt_order = 0

            orders = self.connection.get('fat_pedido','')

            if orders == None:
                print(f'Without orders in database')

                return 
            
            print('*'*100)
            print(f'Name: {name}')
            

            orders = filter_json_dict(orders,'id_cliente',clients[client]['id_cliente'])

            for order in range(len(orders)):
                qt_order += 1

            
            print(f'Qtd order: {qt_order}')
            print('*'*100)

            print('\n')
        return 

    def get_report_product(self,limit=None):
        if self.connection == None:
            print('The connection with database cannot be a None')

            return False 
            
        products = self.connection.get('cad_produto','')

        if products == None:
            print('There arent products in database')

            return False 

        print('*'*100)
        print('* REPORT PRODUCT IN DATABASE')
        print('*'*100)
        print('\n')

        qtd_running = 1

        for i in range(len(products)):
            code = products[i]['cd_produto']
            name = products[i]['nm_produto']
            value_weight = products[i]['vl_peso']
            value_height = products[i]['vl_altura']

            print(f'Code: {code}')
            print(f'Name: {name}')
            print(f'Weight: {value_weight}kg')
            print(f'Height: {value_height}cm')

            print('*'*100)
            print('\n')

            if limit == qtd_running: 
                break

            qtd_running += 1

        return 


## Test area

The script will be connect with the database and get the object CAD_CLIENTE. So will be make a 1 order for each client. 

In [ ]:
try:
    url_database = 'your_firebase_realtime_database'
    database_google = firebase.FirebaseApplication(url_database,None)
except ValueError as e:
    print(e)

# Sáo Paulo 
# Latitude: -23.5475000°
# Longitude: -46.6361100°

#-------------------------------------------------------------------------------
# insert orders
#-------------------------------------------------------------------------------
clients = database_google.get('cad_cliente','')

for i in range(0,1):
    for client in range(len(clients)):
        name = clients[client]['nm_cliente']

        print(f'Get order to {name}')

        insert_order_client(database_google,clients,clients[client]['nr_cpf_cnpj'],[1,2,3,4,5])

        for time_bar in tqdm(range(100)):
            time.sleep(0.1)
#-------------------------------------------------------------------------------

starting_point_coords = (-46.6361100,-23.5475000)

ped,keys = get_orders_delivery(database_google)

if ped and keys:
    rota,pedidos,kilometers_total = set_routing_order(database_google,ped,starting_point_coords)

    truck = Truck('cc')
    today = truck.load_truck_to_delivery(database_google,clients,rota,pedidos,keys)

    if (today.size() > 0) and (truck.truck_is_load()):
        print('\n')
        print('*'*100)
        print('* REPRESENTATION OF TRUCK')
        print('*'*100)
        print('\n')

        print(f'Quantity of clients to visit today: {today.size()}')
        print(f'Route to follow: {rota}')
        print(f'Total Km {kilometers_total}')
        print(f'Orders inside truck: {truck.orders_inside.items}')
        print(f'Weight Total {truck.weight}')
        print(f'Volume Total {truck.volume}')
        
        print('\n')
        print('-'*100)
        print('Truck is going to delivery')
        print('-'*100)

        truck = truck.simulate_delivery(database_google)

    else:
        print('Without route defined!')
    


Get order to Felipe Pedroso de Lima
The order was sent to Delivery Department


100%|██████████| 100/100 [00:10<00:00,  9.74it/s]


Get order to Murilo
The order was sent to Delivery Department


100%|██████████| 100/100 [00:10<00:00,  9.82it/s]


Get order to Ana Laura
The order was sent to Delivery Department


100%|██████████| 100/100 [00:10<00:00,  9.81it/s]


There are 0 to delivery today.


In [ ]:
get_orders_delivery(database_google,'2022-11-27')

There are 3 to delivery today.


({0: {'dt_entrega': '2022-11-27',
   'id_cliente': 1,
   'id_pedido': 182,
   'id_status': 5},
  1: {'dt_entrega': '2022-11-27',
   'id_cliente': 2,
   'id_pedido': 183,
   'id_status': 5},
  2: {'dt_entrega': '2022-11-27',
   'id_cliente': 3,
   'id_pedido': 184,
   'id_status': 5}},
 ['-NHRXFlqc23qaxVhMcOB', '-NHRXIWqeNkNSGfBs1qo', '-NHRXLLkYJo_wcoDpowF'])

In [ ]:
def set_clean_orders_and_items(connection,order_id=None):
    if connection == None:
        return False 

    items = connection.get('fat_pedido_item','')
    orders = connection.get('fat_pedido','')
    
    if order_id != None:
        for i in items:
            if items[i]['id_pedido'] == order_id:
                connection.delete('fat_pedido_item',i)
        
        for k in orders:
            if orders[k]['id_pedido'] == order_id:
                connection.delete('fat_pedido',k)

        return 

    qtd_orders = len(orders)
    qtd_items = len(items)


    ids = []
    keys = []

    loop = 1

    for i in orders:
        if loop == (qtd_orders - 1):
            break 

        ids.append(orders[i]['id_pedido'])
        keys.append(i)

        loop += 1

    ped_item = connection.get('fat_pedido_item','')

    if len(ids) > 0:
        for k in range(len(ids)):
            id_pedido = ids[k]
            
            for j in ped_item:
                if ped_item[j]['id_pedido'] == id_pedido:
                    connection.delete('fat_pedido_item/',j)
                else:
                    continue 

    for p in range(len(keys)):
        connection.delete('fat_pedido/',keys[p])


set_clean_orders_and_items(database_google,183)

In [ ]:
rel = Report(database_google)
rel.get_report_client()

****************************************************************************************************
* REPORT CLIENT IN DATABASE
****************************************************************************************************


****************************************************************************************************
Name: Felipe Pedroso de Lima
Document: 05922356020
Address: Rua Araldo Rodrigues, 13311390 City: 3523909
Coords: (-23.26417, -47.29917)
****************************************************************************************************


****************************************************************************************************
Name: Murilo
Document: 86413288078
Address: Rua Teste, 13310790 City: 3523909
Coords: (-23.50167, -47.45806)
****************************************************************************************************


****************************************************************************************************
Name: Ana Laur